In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

class BoostClass:
    r2 =0
    rmse = 0
    n_estimators = 0
    max_depth = 0
    model = None

    def __init__(self, r2 ,rmse , n_estimators , max_depth,model ):
        self.r2 =r2
        self.rmse = rmse
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.model = model

    def toString(self):
        return "r2: " , self.r2 , ", rmse:" , self.rmse , ", n_estimators:" , self.n_estimators , ", max depth:" , self.max_depth

def fixData(housing):
    """This removes the price column if it exists"""
    if "price" in housing.columns:
        housing = housing.drop(["price"], axis=1)
        
    # Convert the date column to datetime format
    housing['date'] = pd.to_datetime(housing['date'], format='%Y%m%dT%H%M%S')

    housing['day'] = housing['date'].dt.day
    # Calculate the number of days in the current year
    # housing['days_since_april_1st'] = (housing['date'].dt.dayofyear + 365 - 91) % 365

    housing = housing.drop(["id", "date", "day"], axis=1)

    # Add total sqft to housing


    # housing["total_sqft"] = housing["sqft_basement"] + housing["sqft_living"]
    # # housing["age_since_renovation"] = 2025 - housing["yr_renovated"] if housing["yr_renovated"] != 0 else 2025 - housing["yr_built"]
    # # Calculate age since renovation or age since built
    # housing["age_since_renovation"] = housing.apply(lambda row: 2025 - row["yr_renovated"] if row["yr_renovated"] != 0 else 2025 - row["yr_built"], axis=1)

    # scaler = MinMaxScaler(feature_range=(0, 1))

    # for sqft in ['sqft_basement', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_living15', 'sqft_lot15', 'total_sqft']:
    #     housing[sqft] = scaler.fit_transform(housing[[sqft]])


    # turn zip code into category
    # housing["zipcode"] = housing["zipcode"].astype("category")
    # #Turn all categories into dummies
    # housing = pd.get_dummies(housing)

    # do this ^ or that \/ or neither. Not Both!

    # housing = housing.drop("zipcode", axis=1)
    print("COLUMNS: ",housing.columns)

    return housing

In [10]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             20000 non-null  int64  
 1   date           20000 non-null  object 
 2   bedrooms       20000 non-null  int64  
 3   bathrooms      20000 non-null  float64
 4   sqft_living    20000 non-null  int64  
 5   sqft_lot       20000 non-null  int64  
 6   floors         20000 non-null  float64
 7   waterfront     20000 non-null  int64  
 8   view           20000 non-null  int64  
 9   condition      20000 non-null  int64  
 10  grade          20000 non-null  int64  
 11  sqft_above     20000 non-null  int64  
 12  sqft_basement  20000 non-null  int64  
 13  yr_built       20000 non-null  int64  
 14  yr_renovated   20000 non-null  int64  
 15  zipcode        20000 non-null  int64  
 16  lat            20000 non-null  float64
 17  long           20000 non-null  float64
 18  sqft_l

In [3]:
!pip install scikit-learn==1.5.2
# run this line without the '!' in your terminal to have it installed locally
# then you can skip running this



# create days_since_april_1 column
# convert dateTime to days out of 365
# for each value, add 365, subtract april 1st as a number of days
# for each value, mod by 365


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Load dataset
housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
y = np.log1p(housing["price"])  # Log transform target

X = fixData(housing)
print(X.head())

COLUMNS:  Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')
   bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
0         4       3.25         3760      4675     2.0           0     0   
1         3       1.75         1460      7800     1.0           0     0   
2         4       1.00         1340      5800     1.5           0     2   
3         3       1.75         1440      8050     1.0           0     0   
4         2       1.50         1780      4750     1.0           0     0   

   condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
0          3      8        2740           1020      2007             0   
1          2      7        1040            420      1979             0   
2          3      7        1340             

In [5]:
# DO NOT RUN MORE THAN ONCE, we want to keep these variables
boostClassList = []
bestClass = None
# best RMSE so far is ~127000
# 1700 n-estimators, 5 depth

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


# Create XGBoost regressor
for n in range(1):
    for colsample in range(1):
        n = 1742
        d = 6
        model = xgb.XGBRegressor(objective="reg:squarederror", eval_metric="rmse", subsample=0.6,
                            n_estimators=n, max_depth=d, learning_rate=0.01, colsample_bytree=0.7)

        # Train model
        model.fit(X_train, y_train)

        # Make predictions on test set
        y_pred = model.predict(X_test)
        y_non_log_test = np.expm1(y_test)
        y_non_log_pred = np.expm1(y_pred)

        rmse = root_mean_squared_error(y_non_log_test, y_non_log_pred)
        # return from a log of cost to cost
        r2 = r2_score(y_test, y_pred)  # Compute R^2
        r2_non_log = r2_score(y_non_log_test, y_non_log_pred)

        print(f"R² Score: {r2_non_log:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print("estimators: ",n,", depth: ",d)

        boostClass = BoostClass(r2, rmse, n, d, model)
        boostClassList.append(boostClass)
        if (bestClass != None):
            if (boostClass.r2 > bestClass.r2):
                bestClass = boostClass
        else:
            bestClass = boostClass
print("Done!")
# Evaluate performance
print(bestClass.toString())

R² Score: 0.8910
RMSE: 128222.6510
estimators:  1730 , depth:  6
R² Score: 0.8911
RMSE: 128175.4117
estimators:  1731 , depth:  6
R² Score: 0.8910
RMSE: 128193.7961
estimators:  1732 , depth:  6
R² Score: 0.8910
RMSE: 128195.9572
estimators:  1733 , depth:  6
R² Score: 0.8911
RMSE: 128175.0933
estimators:  1734 , depth:  6
R² Score: 0.8911
RMSE: 128176.4253
estimators:  1735 , depth:  6
R² Score: 0.8911
RMSE: 128174.6904
estimators:  1736 , depth:  6
R² Score: 0.8911
RMSE: 128179.9740
estimators:  1737 , depth:  6
R² Score: 0.8911
RMSE: 128159.0499
estimators:  1738 , depth:  6
R² Score: 0.8911
RMSE: 128162.9795
estimators:  1739 , depth:  6
R² Score: 0.8911
RMSE: 128157.9473
estimators:  1740 , depth:  6
R² Score: 0.8911
RMSE: 128152.1404
estimators:  1741 , depth:  6
R² Score: 0.8911
RMSE: 128148.9563
estimators:  1742 , depth:  6
R² Score: 0.8911
RMSE: 128156.5026
estimators:  1743 , depth:  6
R² Score: 0.8911
RMSE: 128155.1492
estimators:  1744 , depth:  6
R² Score: 0.8911
RMSE: 12

In [7]:
# Load the holdout dataset
holdout_data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv')

X_holdout = fixData(holdout_data)

# Predict using the trained model
y_holdout_pred = model.predict(X_holdout)

# Convert predictions back to original scale
y_holdout_pred_original = np.expm1(y_holdout_pred)

# Create DataFrame with correct format
predictions_df = pd.DataFrame({"price": y_holdout_pred_original})


COLUMNS:  Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [8]:

# Save in the required format
team_name = "team8"  # Replace with your actual team name
filename = f"{team_name}-module3-predictions.csv"
predictions_df.to_csv(filename, index=False)

print(f"Predictions saved as {filename}")

Predictions saved as team8-module3-predictions.csv


In [9]:
#Take the predictions on the holdout data set and compare them to the actual prices found in this file
actual_df = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/refs/heads/master/data/housing_holdout_test_mini_answers.csv")
rmse = root_mean_squared_error(actual_df, predictions_df)
r2 = r2_score(actual_df, predictions_df)
print("RMSE: ", rmse, "| r2: ", r2)

RMSE:  99071.30569900123 | r2:  0.8798733984219567
